In [ ]:

import os
import pickle
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense, \
    concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical

# Date preperation

In [ ]:
def process_directory(directory, data_list, file_type):
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)

        # Check if it's a directory
        if os.path.isdir(class_dir):
            # Loop through each image in the folder
            for image_name in os.listdir(class_dir):
                if file_type == 'jpg' and image_name.endswith('.jpg'):
                    image_path = os.path.join(class_dir, image_name)
                    data_list.append({'filepath': image_path, 'label': class_name})
                elif file_type == 'npy' and image_name.endswith('.npy'):
                    image_path = os.path.join(class_dir, image_name)
                    image = np.load(image_path)
                    # Check if the image is of size 128x128 with 8 channels
                    if image.shape == (128, 128, 8):
                        data_list.append({'filepath': image_path, 'label': class_name})

# Initialize an empty list for storing data
data_list = []

# Define paths to your training and testing directories for jpg and npy files
data_dir_jpg = 'Datasets/combined_dataset_processed_128_1'
data_dir_npy = 'Datasets/combined_dataset_enhanced_npy_6_channels'

# Add training images to the data list for jpg and npy files
process_directory(data_dir_jpg, data_list, 'jpg')
process_directory(data_dir_npy, data_list, 'npy')

# Create a DataFrame from the list
df = pd.DataFrame(data_list)

# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

# Print the amount of images per category before balancing
print("Images per category:")
print(df['label'].value_counts())

# Preprocessing

In [ ]:
image_size = (128, 128)
batch_size = 32  # Choose a unified batch size that works well for both datasets

# Split the data into training, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.11, random_state=42)

# Calculate and print split ratios
total_samples = len(df)
train_ratio = len(train_df) / total_samples
val_ratio = len(val_df) / total_samples
test_ratio = len(test_df) / total_samples

print(f"Total samples: {total_samples}")
print(f"Training set: {train_ratio:.2f} ({len(train_df)} samples)")
print(f"Validation set: {val_ratio:.2f} ({len(val_df)} samples)")
print(f"Test set: {test_ratio:.2f} ({len(test_df)} samples)\n")


# Encode labels to integers for .npy data handling
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])
val_df['encoded_label'] = label_encoder.transform(val_df['label'])
test_df['encoded_label'] = label_encoder.transform(test_df['label'])

# Image Data Generators for JPEG images
train_datagen = ImageDataGenerator(
    rescale=1./255, # Normalize pixel values to [0, 1]
    # rotation_range=15,  # rotation. Not needed since all images are getting aligned
    width_shift_range=0.05, # horizontal shift (only 5% since faces are centered)
    height_shift_range=0.05, # vertical shift (only 5% since faces are centered)
    shear_range=0.1, 
    # zoom_range=0.1,   zoom (with current dataset not needed, since faces are centered)
    horizontal_flip=True, # flip images horizontally
    fill_mode='constant', # fill in missing pixels (nearest / constant)
    # brightness_range=[0.8, 1.2] # darken and lighten images
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Prepare Image Generators
train_generator = train_datagen.flow_from_dataframe(train_df[train_df['filepath'].str.contains('.jpg')], x_col='filepath', y_col='label', target_size=image_size, batch_size=batch_size, color_mode='grayscale', class_mode='categorical')
val_generator = val_datagen.flow_from_dataframe(val_df[val_df['filepath'].str.contains('.jpg')], x_col='filepath', y_col='label', target_size=image_size, batch_size=batch_size, color_mode='grayscale', class_mode='categorical')
test_generator = test_datagen.flow_from_dataframe(test_df[test_df['filepath'].str.contains('.jpg')], x_col='filepath', y_col='label', target_size=image_size, batch_size=batch_size, color_mode='grayscale', class_mode='categorical', shuffle=False)

def set_shapes(img, label, img_shape=(128, 128, 8), label_shape=(7, )):
    img.set_shape(img_shape)
    label.set_shape(label_shape)
    return img, label

# Function to preprocess .npy files
def preprocess_npy(file_path, label):
    image = np.load(file_path.numpy())
    image = image.astype(np.float32) / 255.0
    label = to_categorical(label, num_classes=7) 
    return image, label

# TensorFlow Dataset for .npy files
def create_dataset(df):
    df_npy = df[df['filepath'].str.contains('.npy')]
    file_paths = df_npy['filepath'].values
    labels = df_npy['encoded_label'].values
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(lambda x, y: tf.py_function(preprocess_npy, [x, y], [tf.float32, tf.float32]), num_parallel_calls=tf.data.AUTOTUNE)
    #dataset = dataset.map(set_shapes)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)
test_dataset = create_dataset(test_df)

labels = df['label'].values
# Determine unique classes and their corresponding frequencies
classes, frequencies = np.unique(labels, return_counts=True)
# Calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=labels)
# Map the calculated weights to their corresponding class labels
class_weight_dict = {class_label: weight for class_label, weight in zip(classes, class_weights)}
print(class_weight_dict)

for images, labels in train_dataset.take(1):  # Take a single batch from the dataset
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)

# Model Definition

In [ ]:
# Input shapes
input_shape_plain = (128, 128, 1)  # For grayscale images
input_shape_npy = (128, 128, 8)    # For .npy files with 8 channels

# Model configuration
l2_reg = 0.001
dropout_rates = [0.15, 0.5]
n_filters = [128, 64, 64, 32, 32, 128]

# Define two sets of inputs
input_plain = Input(shape=input_shape_plain, name='grayscale_input')
input_npy = Input(shape=input_shape_npy, name='npy_input')

# Grayscale image branch
x1 = Conv2D(n_filters[0], (3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg))(input_plain)
x1 = BatchNormalization()(x1)
x1 = MaxPooling2D(pool_size=(2, 2))(x1)
x1 = Dropout(dropout_rates[0])(x1)

x1 = Conv2D(n_filters[1], (3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg))(x1)
x1 = BatchNormalization()(x1)
x1 = MaxPooling2D(pool_size=(2, 2))(x1)
x1 = Dropout(dropout_rates[0])(x1)

x1 = Conv2D(n_filters[3], (3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg))(x1)
x1 = BatchNormalization()(x1)
x1 = MaxPooling2D(pool_size=(2, 2))(x1)
x1 = Dropout(dropout_rates[0])(x1)

x1 = Flatten()(x1)

# .npy files branch
x2 = Conv2D(n_filters[0], (3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg))(input_npy)
x2 = BatchNormalization()(x2)
x2 = MaxPooling2D(pool_size=(2, 2))(x2)
x2 = Dropout(dropout_rates[0])(x2)

x2 = Conv2D(n_filters[1], (3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg))(x2)
x2 = BatchNormalization()(x2)
x2 = MaxPooling2D(pool_size=(2, 2))(x2)
x2 = Dropout(dropout_rates[0])(x2)

x2 = Conv2D(n_filters[3], (3, 3), padding='same', activation='relu', kernel_regularizer=l2(l2_reg))(x2)
x2 = BatchNormalization()(x2)
x2 = MaxPooling2D(pool_size=(2, 2))(x2)
x2 = Dropout(dropout_rates[0])(x2)

x2 = Flatten()(x2)

# Merge the outputs of the two branches
combined = concatenate([x1, x2])

# Fully connected layers
z = Dense(n_filters[5], activation='relu', kernel_regularizer=l2(l2_reg))(combined)
z = BatchNormalization()(z)
z = Dropout(dropout_rates[1])(z)
z = Dense(7, activation='softmax', kernel_regularizer=l2(l2_reg))(z)  # Assuming 7 classes for output

# Final model
model = Model(inputs=[input_plain, input_npy], outputs=z)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary to verify the architecture
model.summary()

# Callbacks

In [ ]:
# Early stopping to prevent overfitting. This stops training when the model's performance on the validation set starts to degrade.
early_stopper = EarlyStopping(
    monitor='val_loss',  # Metric to be monitored
    patience=3,         # Number of epochs with no improvement after which training will be stopped. Reduced from 10
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored metric
)

# ModelCheckpoint callback
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
checkpoint = ModelCheckpoint(
    f'logs/model_checkpoint_{timestamp}.keras',  # Path where to save the model
    monitor='val_loss',     # Metric to monitor
    save_best_only=False,    # Save only the best model. Set False to save the model at the end of every epoch so restarting from specific epoch is possible
    save_weights_only=False, # Save only the weights
    mode='min',             # Minimize the monitored metric (val_loss) min before
    verbose=1               # Verbose output
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    min_lr=0.0001,
    cooldown=3,
    verbose=1,
    mode='min'
)

# Model Training

In [ ]:
# Custom Data Generator to merge JPEG and NPY datasets
class CustomDataGenerator(Sequence):
    def __init__(self, df_jpg, df_npy, batch_size, image_size, label_encoder, for_training=True):
        self.df_jpg = df_jpg
        self.df_npy = df_npy
        self.batch_size = batch_size
        self.image_size = image_size
        self.label_encoder = label_encoder
        self.for_training = for_training

        # ImageDataGenerator for real-time data augmentation
        self.datagen = ImageDataGenerator(rescale=1./255) if for_training else ImageDataGenerator(rescale=1./255)
        
        self.on_epoch_end()

    def __len__(self):
        # Use the larger of the two datasets to determine the number of steps
        return max(len(self.df_jpg), len(self.df_npy)) // self.batch_size

    def __getitem__(self, index):
        if index % 2 == 0:
            # Process JPEG images
            batch_df = self.df_jpg[index * self.batch_size:(index + 1) * self.batch_size]
            x_batch = np.zeros((len(batch_df), *self.image_size, 1), dtype='float32')
            for i, row in enumerate(batch_df.itertuples()):
                img_path = row.filepath
                img = tf.keras.preprocessing.image.load_img(img_path, color_mode='grayscale', target_size=self.image_size)
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                x_batch[i] = img_array
            
            y_batch = self.label_encoder.transform(batch_df['label'].values)
            y_batch = tf.keras.utils.to_categorical(y_batch, num_classes=7)  # Adjust based on the number of classes
        else:
            # Process NPY files
            batch_df = self.df_npy[index * self.batch_size:(index + 1) * self.batch_size]
            x_batch = np.array([np.load(row.filepath) for row in batch_df.itertuples()])
            y_batch = self.label_encoder.transform(batch_df['label'].values)
            y_batch = tf.keras.utils.to_categorical(y_batch, num_classes=7)  # Adjust based on the number of classes
        
        # Apply data augmentation on-the-fly for JPEG images (if for_training is True)
        if self.for_training and index % 2 == 0:
            x_batch = np.array([self.datagen.random_transform(img) for img in x_batch])
        
        return x_batch, y_batch

    def on_epoch_end(self):
        if self.for_training:
            # Shuffle the dataset (for training only)
            self.df_jpg = self.df_jpg.sample(frac=1).reset_index(drop=True)
            self.df_npy = self.df_npy.sample(frac=1).reset_index(drop=True)

label_encoder = LabelEncoder()
labels = df['label'].values
label_encoder.fit(labels)

class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

# Prepare CustomDataGenerators for training and validation
train_generator = CustomDataGenerator(train_df[train_df['filepath'].str.contains('.jpg')],
                                      train_df[train_df['filepath'].str.contains('.npy')],
                                      batch_size,
                                      image_size,
                                      label_encoder,
                                      for_training=True)
val_generator = CustomDataGenerator(val_df[val_df['filepath'].str.contains('.jpg')],
                                    val_df[val_df['filepath'].str.contains('.npy')],
                                    batch_size,
                                    image_size,
                                    label_encoder,
                                    for_training=False)

# Unified model.fit() call
# history = model.fit(train_generator,
#                     epochs=epochs,
#                     validation_data=val_generator,
#                     class_weight=class_weight_dict,
#                     verbose=1)

# Train the model
history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data=val_generator,
                    class_weight=class_weight_dict,
                    verbose=1,
                    callbacks=callbacks_list,
                    steps_per_epoch=len(train_generator),  # Defined by __len__ in CustomDataGenerator
                    validation_steps=len(val_generator))   # Defined by __len__ in CustomDataGenerator

In [ ]:
# Load the last saved weights
# model.load_weights('logs/model_checkpoint_20240225_085054.keras')

epochs = 30 # When resuming training, set epochs to the total number of epochs you want to train, not just the additional epochs. The model.fit() method continues training for the specified number of epochs, starting from the current epoch count.

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    class_weight=class_weights_dict
    # callbacks=[early_stopper, checkpoint]
)

history_lm = model.fit(
    train_dataset,  # Training data
    epochs=epochs,  # Number of epochs
    validation_data=val_dataset,  # Validation data
    class_weight=class_weight_dict,  # Class weights
    steps_per_epoch=len(train_df) // batch_size,  # Number of steps per epoch, optional if using tf.data.Dataset
    validation_steps=len(val_df) // batch_size,  # Number of validation steps, optional if using tf.data.Dataset
    verbose=1  # Show training log
    # callbacks=[early_stopper, checkpoint]
)

# Save the training history for later analysis
with open(f'logs/training_history_{timestamp}.pkl', 'wb') as file:
    pickle.dump(history.history, file)

# Evaluation and Visualization

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=np.ceil(test_generator.samples / test_generator.batch_size))
print("Test accuracy: ", test_accuracy)

# Predictions on the test set
test_generator.reset() # Ensuring the generator is reset to the beginning
predictions = model.predict(test_generator, steps=np.ceil(test_generator.samples / test_generator.batch_size))
predicted_classes = np.argmax(predictions, axis=1) # Convert predictions to class labels

# Since the generator omits some samples due to rounding down in 'steps', we trim 'true_classes' to match 'predicted_classes' length
true_classes = test_generator.classes
true_classes = true_classes[:len(predicted_classes)]

class_labels = list(test_generator.class_indices.keys())

# Classification report
print("Classification Report")
print(classification_report(true_classes, predicted_classes, target_names=class_labels, zero_division=0))

# Additional weighted metric calculations
weighted_precision = precision_score(true_classes, predicted_classes, average='weighted')
weighted_recall = recall_score(true_classes, predicted_classes, average='weighted')
weighted_f1 = f1_score(true_classes, predicted_classes, average='weighted')

print("Weighted Precision:", weighted_precision)
print("Weighted Recall:", weighted_recall)
print("Weighted F1-Score:", weighted_f1)

# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 10))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

Conducting error analysis
This can be done by examining misclassified examples, which can provide insights into what types of errors the model is making

In [ ]:
# Learning Curves
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Precsion-Recall Curve
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import label_binarize

# Binarize the labels for multi-class
y_bin = label_binarize(true_classes, classes=np.arange(len(class_labels)))
n_classes = y_bin.shape[1]

# Compute precision-recall curve for each class
precision = dict()
recall = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_bin[:, i], predictions[:, i])

# Plot the precision-recall curve
for i in range(n_classes):
    plt.plot(recall[i], precision[i], lw=2, label='Class {}'.format(class_labels[i]))

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="best")
plt.title("Precision vs. Recall curve")
plt.show()

In [ ]:
# ROC Curve and AUC
from sklearn.metrics import roc_curve, auc

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label='Class {} (area = {:.2f})'.format(class_labels[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()